# Export a DISV grid to CSV with the geometry column as EWKT
For each cell, cell identifier, activation status and geometry as EWKT is exported

In [2]:
import flopy.mf6 as mf6
import numpy as np
from pathlib import Path

In [4]:
modelname = 'sc.nam'
model_path = r'E:\LSGB\20231117_JCUNVRS\gwf\gwf3_pest1'
model_path = Path(model_ws)
layer = 0

grid_file = r'E:\LSGB\20231117_JCUNVRS\gwf\calibrations\20240117a\grid.csv'

epsg_code = 25830

# exports only active cells 
exp_only_grid_active = False

In [5]:
# gwf is instantiated and some basic features are shown
sim = mf6.MFSimulation.load(sim_name=modelname, exe_name='mf6', sim_ws=model_path)

model_names = list(sim.model_names)
print('\nModel names in sim', model_names)

gwf = sim.get_model(model_names[0])

# general information of my model
package_list = gwf.get_package_list()
print('Packages:', package_list)
print('Number of layers:', gwf.dis.nlay.array)  # Number of layers
print('Number of cells:', gwf.dis.ncpl.array)  # Number of cells per layer
print('Number of active cells:',(gwf.dis.idomain.array > 0).sum())

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package gnc...
    loading package oc...
    loading package drn...
    loading package rch...
  loading solution package modflow...

Model names in sim ['modflow']
Packages: ['DISV', 'IC', 'NPF', 'OBS-1', 'GNC', 'OC', 'DRN-1', 'RCH-1']
Number of layers: 1
Number of cells: 2331
Number of active cells: 1606


In [6]:
# Get the DISV package
disv = gwf.get_package('disv')

# Get the cell2d array which contains the cell identifiers and vertices
cell2d = disv.cell2d.array
vertices = disv.vertices.array

# get the idomain array
idomain = disv.idomain.array

header="id,active,ewkt"

with open(grid_file, 'w', encoding='utf8') as fo:
    fo.write(f'{header}\n')
    # For each cell
    for ic, cell in enumerate(cell2d):
        lcell = list(cell)
        # Number of vertices for the cell
        num_vertices = lcell[3]
        # Extract the vertex identifiers
        vertex_ids = lcell[4:4 + num_vertices]
        vertex_xy = [list(vertices[v1])[1:] for v1 in vertex_ids]
        # Format coordinates as a string
        coordinate_str = ', '.join([f'{x} {y}' for x, y in vertex_xy])
        # Create EWKT representation
        ewkt_representation = f'SRID={epsg_code};POLYGON(({coordinate_str}))'
        if exp_only_grid_active:
            if idomain[layer,lcell[0]] == 1:
                write_cell= True
            else:
                write_cell = False
        else:
            write_cell = True
        if write_cell:
            fo.write(f'{lcell[0]},{idomain[layer,lcell[0]]},"{ewkt_representation}"\n')
